In [173]:
import http.client as http_client
http_client.HTTPConnection.debuglevel = 0
from watson_developer_cloud import PersonalityInsightsV3, ToneAnalyzerV3, WatsonApiException
import pandas as pd, numpy as np

PersonalityAPI = 'xLGWjB2AUBrzzOoNzXbtB6eucNqqww_28anqfi70YFJ-'
ToneAPI = '-uXV4OpbD7DMEbkqk3UhepcDEDk0oqtYguer2cdHj9x7'

personality_insights = PersonalityInsightsV3(version = '2018-07-13', iam_api_key = PersonalityAPI, url = 'https://gateway-syd.watsonplatform.net/personality-insights/api')
personality_insights.set_default_headers({'x-watson-learning-opt-out': 'true'})

tone_analyzer = ToneAnalyzerV3(version = '2016-05-19', iam_api_key = ToneAPI, url = 'https://gateway-syd.watsonplatform.net/tone-analyzer/api')
tone_analyzer.set_default_headers({'x-watson-learning-opt-out': 'true'})


class Personality():
    '''Get Personality of User'''
    def __init__(self, text):
        self.profile = _GetPersonality(text)
        
    def _GetPersonality(self, text):
        try: return personality_insights.profile(text, content_type = 'text/plain', content_language = 'en', raw_scores = True, consumption_preferences = True)
        except WatsonApiException as ex: print("Method failed with status code " + str(ex.code) + ": " + ex.message); return None;
            
    def _processProfile(self):
        '''------------------First, extract the big 5 personalities and traits.'''
        personalities = []; big_5 = []
        profile = self.profile
        
        for x in profile['personality']:
            traits = pd.DataFrame(x['children'])
            del traits['category']; del traits['trait_id']; del traits['significant'];
            traits['type'] = x['trait_id'].split('_')[1]
            personalities.append(traits)
            big_5_one = pd.DataFrame.from_dict({'name':x['name'], 'percentile':x['percentile'], 'raw_score':x['raw_score'], 'type':'big5'},
                                                orient = 'index').T
            big_5.append(big_5_one)

        personality_profile = pd.concat(personalities)
        personality_profile = pd.concat([personality_profile, pd.concat(big_5)])

        '''------------------Next, Get Needs'''
        needs = pd.DataFrame(profile['needs'])
        needs['type'] = 'needs'; del needs['category']; del needs['significant']; del needs['trait_id'];

        '''------------------Next, Get Values'''
        values = pd.DataFrame(profile['values'])
        values['type'] = 'values'; del values['category']; del values['significant']; del values['trait_id'];

        '''------------------Next, Get consumption_preferences'''
        consume = []
        for x in profile['consumption_preferences']:
            habits = pd.DataFrame(x['consumption_preferences'])
            habits['type'] = 'consume_'+x['name']
            del habits['consumption_preference_id'];
            habits.columns = ['name','raw_score','type']
            consume.append(habits)

        consume_profile = pd.concat(consume)

        '''------------------Merge into 1 large DataFrame'''
        user_profile = pd.concat([personality_profile, needs, values, consume_profile])
        user_profile.reset_index(drop = True, inplace = True)
        
        return user_profile
    
    
class Tone():
    '''Get Tone of text'''
    def __init__(self, text):
        self.tone = self._GetTone(text)
        
    def _GetTone(self, text):
        try: return tone_analyzer.tone(tone_input = text, content_type = 'text/plain', sentences = False, content_language = 'en', tones = ['language'])
        except WatsonApiException as ex: print("Method failed with status code " + str(ex.code) + ": " + ex.message); return None;

    def _processTone(self):
        tone = self.tone
        tones = tone['document_tone']['tone_categories'][0]['tones']
        tones = pd.DataFrame(tones)
        tones['type'] = 'tone'
        tones['name'] = tones['tone_name']
        del tones['tone_id']; del tones['tone_name'];
        tones.columns = ['raw_score','type','name']
        return tones

In [33]:
text = '''pandas.DataFrame.pivot_table
DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')[source]
Create a spreadsheet-style pivot table as a DataFrame. The levels in the pivot table will be stored in MultiIndex objects (hierarchical indexes) on the index and columns of the result DataFrame

Parameters:	
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous

If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

columns : column, Grouper, array, or list of the previous

If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

aggfunc : function, list of functions, dict, default numpy.mean

If list of functions passed, the resulting pivot table will have hierarchical columns whose top level are the function names (inferred from the function objects themselves) If dict is passed, the key is column to aggregate and value is function or list of functions

fill_value : scalar, default None

Value to replace missing values with

margins : boolean, default False

Add all row / columns (e.g. for subtotal / grand totals)

dropna : boolean, default True

Do not include columns whose entries are all NaN

margins_name : string, default ‘All’

Name of the row / column that will contain the totals when margins is True.

Returns:	
table : DataFrame
See also
DataFrame.pivot
pivot without aggregation that can handle non-numeric data'''

In [43]:
profile = GetPersonality(text)

In [170]:
tone = Tone(text)